In [1]:
# CELL 1: Imports & Configuration

import os, json, time
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

# --- CONFIG ---
JSON_PATH = '../data/extracted_legal_data.json'
INDEX_NAME = "legal-assistant"
DIMENSIONS = 384 # Critical for free embeddings

if not os.getenv("PINECONE_API_KEY"): print("❌ Error: Missing Pinecone API Key")
else: print("✅ Ready.")

c:\Users\teju_\OneDrive\Desktop\AI bot\ai_legal_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Ready.


In [ ]:
# CELL 2: Process & Upload (The Workhorse)

# 1. Load Data & Create Docs
if os.path.exists(JSON_PATH):
    with open(JSON_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Convert to LangChain Documents efficiently
    docs = [
        Document(page_content=d.pop('extracted_text'), metadata=d) 
        for d in data if d.get('extracted_text')
    ]
    print(f"📂 Loaded {len(docs)} docs.")
else:
    raise FileNotFoundError("JSON file missing. Run Phase 1 first.")

# 2. Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print(f"✂️ Split into {len(chunks)} chunks.")

# 3. Setup Embeddings & Index
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

if INDEX_NAME not in [i.name for i in pc.list_indexes()]:
    print(f"⚙️ Creating Index '{INDEX_NAME}'...")
    pc.create_index(name=INDEX_NAME, dimension=DIMENSIONS, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))
    time.sleep(10) # Wait for init

# 4. Upload
print(f"🚀 Uploading to Pinecone...")
PineconeVectorStore.from_documents(chunks, embeddings, index_name=INDEX_NAME)
print("✅ SUCCESS! Database updated.")